In [1]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import Dense, Input, Add
from keras.models import Sequential, Model, load_model

Using TensorFlow backend.


In [2]:
def awgn(x_input, reqSNR):
    sig_power = 1
    
    noise_power = np.sqrt(sig_power / (2*reqSNR))
    noise = noise_power * np.random.normal(size = x_input.shape)
    
    return x_input + noise

In [3]:
P = np.array([
    [1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1],
    [1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1],
    [1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1],
    [1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1],
    [1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0],
    [0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0],
    [0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1],
    [1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0],
    [0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0],
    [0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1],
    [1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0],
    [0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1],
], dtype='int')

In [4]:
H = np.concatenate((np.eye(11, dtype="int"), P.T), axis=1)
G= np.concatenate((P, np.eye(12, dtype="int")), axis=1)

In [5]:
codeword_size = 10**6

m = np.random.randint(2, size = (codeword_size, 12))
codeword = np.dot(m,G) % 2
codeword = np.where(codeword > 0 , -1, 1)

In [6]:
Eb_No = 10**(3/10)
Es_No = Eb_No * (12/23)
codeword_noise = awgn(codeword, Es_No)

In [7]:
input_layer = Input(shape = (23,))

line1_Dense1 = Dense(46, activation='elu')(input_layer)
line1_Dense2 = Dense(92, activation='elu')(line1_Dense1)         
line1_Dense3 = Dense(184, activation='elu')(line1_Dense2)
line1_Dense4 = Dense(368, activation='elu')(line1_Dense3)
line1_Dense5 = Dense(184, activation='elu')(line1_Dense4)
line1_Dense6 = Dense(92, activation='elu')(line1_Dense5)
line1_Dense7 = Dense(46, activation='elu')(line1_Dense6)

output_layer = Dense(23, activation='tanh')(line1_Dense7)

model = Model(inputs = input_layer, outputs = output_layer)

model.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics=['accuracy'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [8]:
model.fit(codeword_noise, codeword, 
          epochs=100,
          batch_size=10000,
          shuffle=True)

1000000/1000000 [==============================] - 2s 2us/step - loss: 0.2196 - accuracy: 0.0923
Epoch 38/100
1000000/1000000 [==============================] - 2s 2us/step - loss: 0.2195 - accuracy: 0.0923
Epoch 39/100
1000000/1000000 [==============================] - 2s 2us/step - loss: 0.2192 - accuracy: 0.0933
Epoch 40/100
1000000/1000000 [==============================] - 2s 2us/step - loss: 0.2191 - accuracy: 0.0923
Epoch 41/100
1000000/1000000 [==============================] - 2s 2us/step - loss: 0.2187 - accuracy: 0.0931
Epoch 42/100
1000000/1000000 [==============================] - 2s 2us/step - loss: 0.2183 - accuracy: 0.0929
Epoch 43/100
1000000/1000000 [==============================] - 2s 2us/step - loss: 0.2179 - accuracy: 0.0919
Epoch 44/100
1000000/1000000 [==============================] - 2s 2us/step - loss: 0.2175 - accuracy: 0.0913
Epoch 45/100
1000000/1000000 [==============================] - 2s 2us/step - loss: 0.2172 - accuracy: 0.0915
Epoch 46/100
1000000/10

In [9]:
model.save('golay_ex.h5')